In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# 현재 날짜 및 날짜 계산
end_date = datetime.now()
# 시작 날짜: 오늘 기준으로 370일 전
start_date = end_date - timedelta(days=370)

# S&P 500 구성 종목 티커 불러오기
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

# SYMBOL로 정의
SYMBOL = sp500_tickers

# .을 -로 대체
SYMBOL = [ticker.replace('.', '-') for ticker in SYMBOL]

# DataFrame 생성
df = pd.DataFrame(SYMBOL, columns=['Ticker'])

# Excel 파일 경로
file_path = r"C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx"
df.to_excel(file_path, index=False, engine='openpyxl')  # 티커 목록 저장

print(f"'{file_path}'에 저장되었습니다.")

# 엑셀 파일에서 티커 읽기
df_tickers = pd.read_excel(file_path, sheet_name=0)  # 첫 번째 시트에서 티커 불러오기
tickers = df_tickers['Ticker'].tolist()  # 'Ticker' 열로 티커 목록 가져오기

# 빈 DataFrame 생성 (종가, 거래량, 시가, 고가, 저가에 대한)
price_data = []
volume_data = []
open_data = []
high_data = []
low_data = []

# 데이터 가져오기
for ticker in SYMBOL:
    try:
        # 최근 370일 데이터 가져오기
        data = yf.download(ticker, start=start_date, end=end_date, interval='1d', progress=False)

        if not data.empty:
            # 시간대 정보 제거
            data.index = pd.to_datetime(data.index).tz_localize(None)
            
            price_data.append(data['Close'])  # 종가를 리스트에 추가
            volume_data.append(data['Volume'])  # 거래량을 리스트에 추가
            open_data.append(data['Open'])      # 시가를 리스트에 추가
            high_data.append(data['High'])      # 고가를 리스트에 추가
            low_data.append(data['Low'])        # 저가를 리스트에 추가
        else:
            print(f"{ticker} 데이터가 없습니다.")
    except Exception as e:
        print(f"{ticker} 데이터를 불러오는 중 오류 발생: {e}")

# 가격 데이터가 비어있지 않으면 엑셀에 저장
if price_data:
    # 모든 종가 데이터를 하나의 DataFrame으로 병합
    price_data_df = pd.concat(price_data, axis=1)  # 종가 데이터 DataFrame 생성
    price_data_df.columns = SYMBOL  # 컬럼 이름을 티커로 설정

    # Excel 파일 경로
    file_path = r"C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx"
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        # '종가' 시트에 저장
        price_data_df.to_excel(writer, sheet_name='종가', index=True)  
        print(f"'{file_path}'에 '종가' 시트에 데이터가 저장되었습니다.")

if volume_data:
    # 모든 거래량 데이터를 하나의 DataFrame으로 병합
    volume_data_df = pd.concat(volume_data, axis=1)  # 거래량 데이터 DataFrame 생성
    volume_data_df.columns = SYMBOL  # 컬럼 이름을 티커로 설정

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        # '거래량' 시트에 저장
        volume_data_df.to_excel(writer, sheet_name='거래량', index=True)  
        print(f"'{file_path}'에 '거래량' 시트에 데이터가 저장되었습니다.")

if open_data:
    open_data_df = pd.concat(open_data, axis=1)  # 시가 데이터 DataFrame 생성
    open_data_df.columns = SYMBOL  
    
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        open_data_df.to_excel(writer, sheet_name='시가', index=True)  
        print(f"'{file_path}'에 '시가' 시트에 데이터가 저장되었습니다.")

if high_data:
    high_data_df = pd.concat(high_data, axis=1)  # 고가 데이터 DataFrame 생성
    high_data_df.columns = SYMBOL  
    
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        high_data_df.to_excel(writer, sheet_name='고가', index=True)  
        print(f"'{file_path}'에 '고가' 시트에 데이터가 저장되었습니다.")

if low_data:
    # 저가 데이터를 하나의 DataFrame으로 병합
    low_data_df = pd.concat(low_data, axis=1)  # 저가 데이터 DataFrame 생성
    low_data_df.columns = tickers  # 컬럼 이름을 티커로 설정

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
        low_data_df.to_excel(writer, sheet_name='저가', index=True)  # '저가' 시트에 저장
        print(f"'{file_path}'에 '저가' 시트에 데이터가 저장되었습니다.")

'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 저장되었습니다.
'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 '종가' 시트에 데이터가 저장되었습니다.
'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 '거래량' 시트에 데이터가 저장되었습니다.
'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 '시가' 시트에 데이터가 저장되었습니다.
'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 '고가' 시트에 데이터가 저장되었습니다.
'C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx'에 '저가' 시트에 데이터가 저장되었습니다.


In [2]:
import pandas as pd

# 엑셀 파일 경로
file_path = r"C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx"

# 종가 데이터 읽기
df_close = pd.read_excel(file_path, sheet_name='종가', index_col=0, parse_dates=True)  # 'Date' 열을 인덱스로 설정

# 수익률 계산 함수
def calculate_returns(data, period):
    if period == '1w' and len(data) >= 5:
        return (data.iloc[-1] / data.iloc[-5]) - 1  # 1주 전과 오늘 비교
    elif period == '1mo' and len(data) >= 21:
        return (data.iloc[-1] / data.iloc[-21]) - 1  # 1달 전과 오늘 비교
    elif period == '3mo' and len(data) >= 63:
        return (data.iloc[-1] / data.iloc[-63]) - 1  # 3달 전과 오늘 비교
    elif period == '6mo' and len(data) >= 126:
        return (data.iloc[-1] / data.iloc[-126]) - 1  # 6개월 전과 오늘 비교
    elif period == '1y' and len(data) >= 252:
        return (data.iloc[-1] / data.iloc[-252]) - 1  # 1년 전과 오늘 비교
    return None  # 데이터가 충분하지 않으면 None

# 수익률을 저장할 DataFrame 생성
returns = pd.DataFrame(index=df_close.columns)

# 각 기간별 수익률 계산
for period in ['1w', '1mo', '3mo']:
    returns[period] = [calculate_returns(df_close[ticker], period) for ticker in df_close.columns]

# NaN 값 처리
returns = returns.apply(pd.to_numeric, errors='coerce')  # NaN 처리

# 각 기간별 상위 15개 종목 묶어서 출력
for period in ['1w', '1mo', '3mo']:
    top_15 = returns[period].nlargest(15)  # 상위 15개 종목
    print(f"\n{period} 기간 수익률 상위 15개 종목:")
    for ticker, momentum in top_15.items():
        print(f"{ticker}: {momentum * 100:.2f}%")  # 수익률을 백분율로 출력



1w 기간 수익률 상위 15개 종목:
AXON: 28.28%
FTNT: 16.97%
TAP: 12.60%
TSLA: 11.23%
CRM: 9.96%
PODD: 9.83%
PLTR: 8.63%
ALB: 8.27%
FICO: 7.92%
TSN: 7.82%
PYPL: 7.33%
HON: 7.16%
CPAY: 7.00%
SCHW: 6.94%
LNT: 6.93%

1mo 기간 수익률 상위 15개 종목:
TSLA: 49.21%
PLTR: 44.77%
PAYC: 39.27%
AXON: 38.60%
GRMN: 27.46%
EPAM: 24.69%
UAL: 24.66%
CHTR: 23.01%
MCK: 22.25%
INCY: 21.63%
WBD: 21.50%
GEV: 21.29%
SNA: 21.20%
DAY: 20.65%
EXPE: 19.55%

3mo 기간 수익률 상위 15개 종목:
UAL: 113.41%
PLTR: 89.21%
GEV: 81.06%
VST: 79.11%
NCLH: 69.67%
AXON: 60.91%
DAL: 60.55%
CCL: 58.80%
TSLA: 52.80%
RCL: 48.45%
PAYC: 42.38%
SYF: 39.70%
DAY: 39.30%
PODD: 39.04%
RJF: 38.54%


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# 현재 날짜 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1년 전 날짜

# 엑셀 파일 경로
file_path = r"C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx"

# 종가 데이터 읽기
df_close = pd.read_excel(file_path, sheet_name='종가', index_col=0, parse_dates=True)

# 수익률 계산 함수
def calculate_returns(data):
    return data.pct_change()  # 일일 수익률 계산

# 주간 변동성 계산 함수
def calculate_weekly_volatility(data):
    weekly_returns = data.resample('W').ffill().pct_change()  # 주간 수익률 계산
    return weekly_returns.std()  # 주간 수익률의 표준편차 (변동성) 계산

# 변동성을 저장할 DataFrame 정의
var_data = pd.DataFrame(index=df_close.columns, columns=['Var'])

# 각 종목의 주간 변동성 계산 및 저장
for ticker in df_close.columns:
    var_data.loc[ticker, 'Var'] = calculate_weekly_volatility(df_close[ticker])

# 결과 출력
print("주간 변동성 (Var):")
print(var_data)



주간 변동성 (Var):
           Var
MMM   0.045242
AOS   0.032955
ABT   0.025389
ABBV  0.034877
ACN   0.033147
...        ...
XYL   0.029618
YUM    0.02377
ZBRA   0.04378
ZBH   0.027997
ZTS   0.033315

[503 rows x 1 columns]


In [4]:
pip install mplfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import mplfinance as mpf
import matplotlib.pyplot as plt
import os
from datetime import datetime

# 현재 날짜 및 날짜 계산
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1년 전 날짜

# 엑셀 파일 경로
file_path = r"C:\Users\infomax\Desktop\운용\유니버스\S&P500.xlsx"

# 종가 데이터 읽기
df_close = pd.read_excel(file_path, sheet_name='종가', index_col=0, parse_dates=True)

# 모멘텀 계산 함수
def calculate_momentum(data):
    momentum = {}
    # 각 기간에 대한 수익률 계산
    for period, months in {'1m': 21, '3m': 63, '6m': 126, '1y': 252}.items():
        if len(data) >= months:  # 데이터를 충분히 가지고 있는지 확인
            returns = (data.iloc[-1] / data.iloc[-months]) - 1  # 수익률 계산
            momentum[period] = returns
        else:
            momentum[period] = None  # 데이터가 충분하지 않으면 None
    
    # 모멘텀 계산: (1m+3m+6m+1y)/4
    if all(v is not None for v in momentum.values()):
        momentum_value = sum(momentum.values()) / 4
        return momentum_value
    else:
        return None

# 변동성 계산 함수
def calculate_volatility(data):
    weekly_returns = data.resample('W').ffill().pct_change()  # 주간 수익률 계산
    return weekly_returns.std()  # 주간 수익률의 표준편차 (변동성) 계산

# 모멘텀과 변동성을 저장할 DataFrame 생성
momentum_data = pd.DataFrame(index=df_close.columns, columns=['Momentum'])
volatility_data = pd.DataFrame(index=df_close.columns, columns=['Var'])

# 모멘텀 및 변동성 계산
for ticker in df_close.columns:
    momentum_value = calculate_momentum(df_close[ticker])
    volatility_value = calculate_volatility(df_close[ticker])
    
    momentum_data.loc[ticker, 'Momentum'] = momentum_value
    volatility_data.loc[ticker, 'Var'] = volatility_value

# NaN 값을 처리하여 수치형 데이터로 변환
momentum_data['Momentum'] = pd.to_numeric(momentum_data['Momentum'], errors='coerce')  # NaN 처리
volatility_data['Var'] = pd.to_numeric(volatility_data['Var'], errors='coerce')  # NaN 처리

# 모멘텀과 변동성의 비율 계산 (Momentum / Var)
combined_data = momentum_data.join(volatility_data)
combined_data['Momentum/Var'] = combined_data['Momentum'] / combined_data['Var']

# 상위 30개 종목(모멘텀/변동성 비율 기준으로) 선택
top_30_momentum_var = combined_data['Momentum/Var'].nlargest(30)  # 상위 30개 종목

# 상위 30개 종목 티커 리스트 생성
tickers_top_30 = top_30_momentum_var.index.tolist()
print("상위 30개 종목 티커 리스트:")
print(tickers_top_30)

# 종가 데이터 읽기
df_close = pd.read_excel(file_path, sheet_name='종가', index_col=0, parse_dates=True)
# 고가 데이터 읽기
df_high = pd.read_excel(file_path, sheet_name='고가', index_col=0, parse_dates=True)
# 저가 데이터 읽기
df_low = pd.read_excel(file_path, sheet_name='저가', index_col=0, parse_dates=True)
# 시가 데이터 읽기
df_open = pd.read_excel(file_path, sheet_name='시가', index_col=0, parse_dates=True)
# 거래량 데이터 읽기
df_volume = pd.read_excel(file_path, sheet_name='거래량', index_col=0, parse_dates=True)

# 차트 저장 디렉토리 설정
chart_directory = r"C:\Users\infomax\Desktop\운용\유니버스\차트"
os.makedirs(chart_directory, exist_ok=True)  # 차트 폴더가 없으면 생성

# 오류 저장 리스트 초기화
errors = []

# 종가와 거래량 데이터 가져오기
for ticker in tickers_top_30:
    try:
        # 필요한 데이터 프레임 생성
        data = pd.DataFrame({
            'Open': df_open[ticker],
            'High': df_high[ticker],
            'Low': df_low[ticker],
            'Close': df_close[ticker],
            'Volume': df_volume[ticker]
        })

        # 범위가 최근 일자까지 포함되어 있는지 점검
        data = data.loc[start_date:end_date]

        # 캔들차트 그리기 및 저장
        mpf.plot(data, type='candle', volume=True, style='charles',
                 title=f"{ticker} chart", ylabel='Price',
                 ylabel_lower='Volume',
                 figratio=(18, 10), figscale=1.5,
                 savefig=os.path.join(chart_directory, f"{ticker}.jpeg"))
    except Exception as e:
        errors.append((ticker, str(e)))

if errors:
    print("다음 종목의 캔들차트를 만드는 중 오류가 발생했습니다:")
    for ticker, error in errors:
        print(f"{ticker}: {error}")
else:
    print("모든 캔들차트가 성공적으로 저장되었습니다.")


상위 30개 종목 티커 리스트:
['FI', 'TRGP', 'TMUS', 'LDOS', 'RCL', 'VST', 'FICO', 'UAL', 'KKR', 'PLTR', 'GDDY', 'AXON', 'BK', 'MSI', 'FOX', 'SYF', 'ETR', 'FOXA', 'HWM', 'KMI', 'WMT', 'WELL', 'FFIV', 'WMB', 'BSX', 'ISRG', 'DAL', 'CTAS', 'TT', 'OKE']
모든 캔들차트가 성공적으로 저장되었습니다.
